## 지오코딩(geocoding)
주소를 이용해서 좌표 속성 또는 포인트 geometry를 구하는 기법을 말합니다.  
주소 외에도 지명, 도로명, 우편번호, 행정구역명 등도 포함시켜 볼 수 있습니다. 

https://developers.google.com/maps/documentation/geocoding/start 

미리 말씀드렸던 Naver Geocoding API 신청이 필요합니다.     
https://wooiljeong.github.io/python/geocoding/             # 네이버 지오코딩 API 이용 관련 최신 블로그   
https://www.ncloud.com/product/applicationService/maps     # Maps API 홈페이지     
https://console.ncloud.com/naver-service/application       # Naver Cloud Platform - Application 정보 확인 페이지  

### 패키지 불러오기

In [1]:
import pandas as pd
import geopandas as gpd

c:\Users\thlee\AppData\Local\Programs\Python\Python39\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.10.1-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


### 데이터 로딩

In [33]:
# "전국CCTV표준데이터" CSV 파일 로딩 
df_cctv = pd.read_csv('data/csv/전국CCTV표준데이터.csv', encoding="EUC-KR", header=0, index_col=None,
                           names=['admin_nm', 'addr_rd', 'addr_jb', 'purpose', 'cctv_cnt', 'pixel', 'direction', 'period', 'date_install', 'tel', 'lat', 'lon', 'date_make', 'offer_admin_cd', 'offer_admin_nm'], 
                           dtype={'cctv_cnt':object, 'pixel':object, 'period':object, 'lat':float, 'lon':float, 'offer_admin_cd':object}, 
                  thousands = ',', #천단위 쉼표 제거   
                  nrows=100) # 1백건만
df_cctv.head()

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm
0,경상남도 김해시청,NaN,경상남도 김해시 구산동 1072-19,생활방범,3,200,현대병원뒤,30,2019-11,055-330-4741,35.249247,128.871639,2019-06-26,5350000,경상남도 김해시
1,경상남도 김해시청,NaN,경상남도 김해시 구산동 305-15,생활방범,2,200,목화골공원,30,2019-12,055-330-4741,35.247411,128.873605,2019-06-26,5350000,경상남도 김해시
2,경상남도 김해시청,NaN,경상남도 김해시 삼계동 1484-14,생활방범,2,200,정원빌라,30,2019-06,055-330-4741,35.261728,128.874144,2019-06-26,5350000,경상남도 김해시
3,경상남도 김해시청,NaN,경상남도 김해시 구산동 1043,생활방범,4,200,최가아구찜,30,2019-05,055-330-4741,35.247076,128.872461,2019-06-26,5350000,경상남도 김해시
4,경상남도 김해시청,NaN,경상남도 김해시 구산동 175-10,생활방범,2,200,동호맨션,30,2019-12,055-330-4741,35.241185,128.877334,2019-06-26,5350000,경상남도 김해시


In [25]:
# cctv 데이터프레임의 항목별 결측치(null) 건수를 확인
df_cctv.isnull().sum()

admin_nm           0
addr_rd           71
addr_jb            0
purpose            0
cctv_cnt           0
pixel             14
direction          0
period             0
date_install       2
tel                0
lat                0
lon                0
date_make          0
offer_admin_cd     0
offer_admin_nm     0
dtype: int64

In [34]:
# 지번주소에서 결측치를 제거 
df_cctv.dropna(subset=['addr_jb'],axis=0,inplace=True)      # 지번주소 null 데이터 삭제 
#df_cctv2 = df_cctv.loc[ df_cctv.notnull()['addr_jb'] , :]   # addr_jb 항목이 null 아닌 데이터만 추출해서 다른 데이터프레임으로 저장 

## 이제 지오코딩을 해봅시다

In [9]:
# 웹과 통신해서 지오코딩을 요청하고, 그 결과 데이터를 json으로 처리하기 위한 패키지 import 
import urllib       # 웹상의 데이터를 가져오는 등의 기능을 제공해주는 패키지 
import json         # json 가공 처리 기능을 제공해주는 패키지 

In [10]:
# 네이버 API 정보 읽기 
# 저는 아래 경로의 텍스트 파일 1행에 api_id, 2행에 api_key를 넣어놨습니다. 
f = open('../../etc/naver.txt', 'r')
lines = f.readlines()
api_id = lines[0].replace("\n", "")
api_key = lines[1].replace("\n", "")

# 파일로 관리할 필요가 없는 분들은 아래와 같이 문자열 변수로 적용해 주시면 됩니다. 
# api_id = '아이디';    # 본인이 할당받은 ID 입력
# api_key = '비밀번호';    # 본인이 할당받은 Secret 입력 

In [13]:
# 네이버 지오코딩 API 기능을 search_map 함수로 생성 
def search_map(search_text):
    client_id = 'id'
    client_secret = 'secret'
    encText = urllib.parse.quote(str(search_text))
    url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='+encText
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', api_id) 
    request.add_header('X-NCP-APIGW-API-KEY', api_key) 
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        #print(response_body.decode('utf-8'))
        return response_body.decode('utf-8')
    else:
        print("Error Code:" + rescode)

In [14]:
# 단 건 테스트 - 자기 집 주소를 넣어서 좌표가 제대로 나오는지 확인해 보십시오
x = []
y = []

temp_map = search_map('서울특별시 종로구 청와대로 1')  #NAVER search_map 함수 실행
temp_map = json.loads(temp_map)
try:
    #print(temp_map)
    temp_map = temp_map['addresses'][0]
    x = temp_map['x']
    y = temp_map['y']
except IndexError:
    pass

print(x, y)

126.9748377 37.5864576


In [16]:
# 위의 결과로 나온 경도, 위도 좌표를 아래의 x, y 파라미터에 넣어서 웹브라우저에서 확인해 보십시오     
# https://epsg.io/map#srs=4326&x=126.974838&y=37.586458&z=17&layer=streets 

## 이제 데이터프레임의 여러 건의 주소를 지오코딩해봅시다

In [22]:
# 다수의 지오코딩 결과가 cell output로 계속 나오는 것을 막을 수 있게 해주는 패키지입니다. 
# !pip install etils  #No module named 'etils' 에러나는 분은 왼쪽의 패키지 설치 구문을 주석 해제해서 설치하고 import 해주세요
from etils import ecolab  #cell output 숨기기

In [35]:
x1 = []     #지오코딩 결과의 경도 좌표를 받을 리스트 
y1 = []     #지오코딩 결과의 위도 좌표를 받을 리스트 
length_df = len(df_cctv)  #데이터 건수를 변수에 받음 - 아래의 반복문을 돌리기 위한 용도
print(length_df)

100


In [36]:
# 데이터프레임 행수만큼 읽으면서 addr 항목의 주소를 Naver API로 돌려서 좌표값을 받아 리스트에 추가  
with ecolab.collapse(): 

    for i in range(length_df):              #데이터의 건수
        addr = df_cctv.iloc[i]['addr_jb']   #데이터프레임과 항목(컬럼) 지정 
        #print(addr)
        #print(i)
        temp_map = search_map(addr)  #NAVER search_map 함수 실행
        temp_map = json.loads(temp_map)
        try:
            #print(temp_map)
            temp_map = temp_map['addresses'][0]
            x = temp_map['x']
            y = temp_map['y']
            x1.append(x)
            y1.append(y)    
        except:
            x1.append(0)
            y1.append(0)   

In [37]:
# 좌표 리스트 값을 원래의 데이터프레임의 lon, lat 항목에 넣어줌 
df_cctv['lon2'] = x1
df_cctv['lat2'] = y1

In [39]:
df_cctv.info()
df_cctv.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   admin_nm        100 non-null    object 
 1   addr_rd         29 non-null     object 
 2   addr_jb         100 non-null    object 
 3   purpose         100 non-null    object 
 4   cctv_cnt        100 non-null    object 
 5   pixel           86 non-null     object 
 6   direction       100 non-null    object 
 7   period          100 non-null    object 
 8   date_install    98 non-null     object 
 9   tel             100 non-null    object 
 10  lat             100 non-null    float64
 11  lon             100 non-null    float64
 12  date_make       100 non-null    object 
 13  offer_admin_cd  100 non-null    object 
 14  offer_admin_nm  100 non-null    object 
 15  lon2            100 non-null    object 
 16  lat2            100 non-null    object 
dtypes: float64(2), object(15)
memory usa

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm,lon2,lat2
0,경상남도 김해시청,NaN,경상남도 김해시 구산동 1072-19,생활방범,3,200,현대병원뒤,30,2019-11,055-330-4741,35.249247,128.871639,2019-06-26,5350000,경상남도 김해시,128.8716256,35.2496502
1,경상남도 김해시청,NaN,경상남도 김해시 구산동 305-15,생활방범,2,200,목화골공원,30,2019-12,055-330-4741,35.247411,128.873605,2019-06-26,5350000,경상남도 김해시,128.8734019,35.2473902
2,경상남도 김해시청,NaN,경상남도 김해시 삼계동 1484-14,생활방범,2,200,정원빌라,30,2019-06,055-330-4741,35.261728,128.874144,2019-06-26,5350000,경상남도 김해시,128.8741368,35.2618309
3,경상남도 김해시청,NaN,경상남도 김해시 구산동 1043,생활방범,4,200,최가아구찜,30,2019-05,055-330-4741,35.247076,128.872461,2019-06-26,5350000,경상남도 김해시,128.8718889,35.2464445
4,경상남도 김해시청,NaN,경상남도 김해시 구산동 175-10,생활방범,2,200,동호맨션,30,2019-12,055-330-4741,35.241185,128.877334,2019-06-26,5350000,경상남도 김해시,128.8773246,35.2411248


In [40]:
# 좌표 항목을 실수 형식으로 변환 
df_cctv['lon2'] = df_cctv['lon2'].astype(float)
df_cctv['lat2'] = df_cctv['lat2'].astype(float)

In [41]:
# 좌표를 이용하여 지오데이터프레임으로 생성 
geom = gpd.points_from_xy(df_cctv.lon2, df_cctv.lat2)
cctv_gdf = gpd.GeoDataFrame(df_cctv, geometry=geom, crs=4326)

In [42]:
cctv_gdf.explore(color='red', tiles='http://basemaps.cartocdn.com/dark_all/{z}/{x}/{y}.png', attr='Carto')

In [43]:
# 공간데이터 파일로 저장 - result 폴더가 있어야 합니다 
cctv_gdf.to_file('result/cctv.shp', encoding='cp949')

C:\Users\thlee\AppData\Local\Temp\ipykernel_29468\1546033548.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  cctv_gdf.to_file('result/cctv.shp', encoding='cp949')


## 수고많으셨습니다!!!